In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
%pylab inline

import datetime
import devlib
import os
import json
import pandas as pd
import re
import subprocess
import trappy
from trappy.plotter.Utils import get_trace_event_data

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Support to access the remote target
#import devlib
#from env import TestEnv

from executor import Executor

Populating the interactive namespace from numpy and matplotlib


# Target Configuration

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'salvator-x',
#   "board"       : 'juno',
    
    # Target board IP/MAC address
    "host"        : '10.0.0.57',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'insecure',

}

# Tests Configuration

In [4]:
my_tests_conf = {
    
    #/* Devlib modules to enable/disbale for all the experiments */
    "modules"         : [ "cpufreq", "cgroups" ],
    "exclude_modules" : [ "hwmon" ], # Disable hwmon - no DVFS

    #/* Binary tools required by the experiments */
    "tools"    : [ "rt-app" ],

    #/* FTrace configuration */
    "ftrace" : {
        "events" : [
            "sched_switch",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_boostgroup_update",
            "sched_tune_filter",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_energy_diff",
            "cpu_frequency",
            "cpu_capacity",
            "cpu_idle",# Add cpuidle trace
            "sched_overutilized"
        ],
        "buffsize" : 10240,
    },

    #/* Set of platform configurations to test */
    "confs" : [
        {
            "tag" : "noboost",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" :  0 },
                    }
                },
                "default" : "/",
            }
        },
        {
            "tag" : "boost15",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" : 15 },
                    }
                },
                "default" : "/stune",
            }
        },
        {
            "tag" : "boost30",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" : 30 },
                    }
                },
                "default" : "/stune",
            }
        },
        {
            "tag" : "boost60",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" : 60 },
                    }
                },
                "default" : "/stune",
            }
        }

    ],

    #/* Set of workloads to run on each platform configuration */
    "wloads" : {
        "mixprof" : {
            "type": "rt-app",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "r5_10-60" : {
                        "kind"   : "Ramp",
                        "params" : {
                            "period_ms" : 16,
                            "start_pct" :  5,
                            "end_pct"   : 60,
                            "delta_pct" :  5,
                            "time_s"    :  1,
                         }
                    }
                }
            },
            "loadref" : "LITTLE",
        }
    },

    #/* Number of iterations for each workload */
    "iterations" : 1,

}

# Tests execution

In [5]:
executor = Executor(my_target_conf, my_tests_conf)

03:22:30  INFO    :         Target - Loading custom (inline) test configuration
03:22:30  INFO    :         Target - Using base path: /home/ed/ar001/lisa
03:22:30  INFO    :         Target - Loading custom (inline) target configuration
03:22:30  INFO    :         Target - Loading custom (inline) test configuration
03:22:30  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'cgroups']
03:22:30  INFO    :         Target - Connecting linux target:
03:22:30  INFO    :         Target -   username : root
03:22:30  INFO    :         Target -       host : 10.0.0.57
03:22:30  INFO    :         Target -   password : insecure
2016-06-20 15:22:30,759 DEBUG   : Logging in root@10.0.0.57
2016-06-20 15:22:32,744 DEBUG   : id
2016-06-20 15:22:33,174 DEBUG   : if [ -e '/root/devlib-target/bin' ]; then echo 1; else echo 0; fi
2016-06-20 15:22:33,586 DEBUG   : ls -1 /root/devlib-target/bin
2016-06-20 15:22:34,040 DEBUG   : cat /proc/cpuinfo
2016-06-20 15:22:34,460 DEBUG   : sudo -- sh 

In [6]:
executor.run()

03:23:24  INFO    : 
03:23:24  INFO    : ################################################################################
03:23:24  INFO    :       Executor - Experiments execution
03:23:24  INFO    : ################################################################################
03:23:24  INFO    : 
03:23:24  INFO    : ================================================================================
03:23:24  INFO    :   TargetConfig - configuring target for [noboost] experiments
03:23:24  INFO    :        CGroups - Initialize CGroups support...
03:23:24  INFO    :        CGroups - Setup [schedtune] controller...
2016-06-20 15:23:24,160 DEBUG   : sudo -- sh -c '[ -d /root/devlib-target/run_dir ] || mkdir /root/devlib-target/run_dir'
2016-06-20 15:23:25,067 DEBUG   : sudo -- sh -c 'grep schedtest /proc/mounts ||   mount -t tmpfs -o size=1024m schedtest /root/devlib-target/run_dir'
03:23:25  INFO    :  SchedFeatures - Set scheduler feature: ENERGY_AWARE
2016-06-20 15:23:25,978 DEBUG   :

In [7]:
!tree {executor.te.res_dir}

/home/ed/ar001/lisa/results/20160620_152324
├── rtapp:boost15:mixprof
│   ├── 1
│   │   ├── mixprof_00.json
│   │   ├── output.log
│   │   ├── rt-app-task_r5_10-60-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:boost30:mixprof
│   ├── 1
│   │   ├── mixprof_00.json
│   │   ├── output.log
│   │   ├── rt-app-task_r5_10-60-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:boost60:mixprof
│   ├── 1
│   │   ├── mixprof_00.json
│   │   ├── output.log
│   │   ├── rt-app-task_r5_10-60-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
└── rtapp:noboost:mixprof
    ├── 1
    │   ├── mixprof_00.json
    │   ├── output.log
    │   ├── rt-app-task_r5_10-60-0.log
    │   └── trace.dat
    ├── kernel.config
    ├── kernel.version
    └── platform.json

8 directories, 28 files
